# Day 5

## part 1

- the first section lists some ordering info `X|Y` means `X` must be before `Y`
- the second section has updates which must be checked against the ordering info.
- The elves need the middle number of correctly ordered updates
- Find their sum

In [1]:
from dataclasses import dataclass
import logging

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig as PC, markdown
)

log = logging.getLogger('day 5')
logging.basicConfig(level=logging.INFO)

In [2]:
parser = PC('\n\n', [
    PC('\n', PC('|', int)),  # first section
    PC('\n', PC(',', int)),  # second section
])

example_ordering, example_updates = \
    parse_from_file('day_5_example.txt', parser)

log.info(f'{example_ordering=}')
log.info(f'{example_updates=}')

INFO:advent_of_code_utils.py:2 items loaded from "day_5_example.txt"
INFO:day 5:example_ordering=[[47, 53], [97, 13], [97, 61], [97, 47], [75, 29], [61, 13], [75, 53], [29, 13], [97, 29], [53, 29], [61, 53], [97, 53], [61, 29], [47, 13], [75, 47], [97, 75], [47, 61], [75, 61], [47, 29], [75, 13], [53, 13]]
INFO:day 5:example_updates=[[75, 47, 61, 53, 29], [97, 61, 53, 29, 13], [75, 29, 13], [75, 97, 47, 61, 53], [61, 13, 29], [97, 13, 75, 29, 47]]


In [3]:
# cool the parser config works so lets test out a solution
@dataclass
class Rule:
    before: int
    after: int

def get_rules(ordering: list[list[int]]) -> dict[int: Rule]:
    """returns a searchable dict of rule objects to look up by number"""
    rules = {}
    for before, after in ordering:
        for key in (before, after):
            if key not in rules:
                rules.update({key: [Rule(before, after)]})
                log.debug(f'Added new {key=}')
            else:
                rules[key].append(Rule(before, after))
            log.debug(f'New rule: {rules[key][-1]}')
    log.info(f'created rules lookup for {len(rules)} values!')
    return rules

log.setLevel(logging.DEBUG)
example_rules = get_rules(example_ordering)

DEBUG:day 5:Added new key=47
DEBUG:day 5:New rule: Rule(before=47, after=53)
DEBUG:day 5:Added new key=53
DEBUG:day 5:New rule: Rule(before=47, after=53)
DEBUG:day 5:Added new key=97
DEBUG:day 5:New rule: Rule(before=97, after=13)
DEBUG:day 5:Added new key=13
DEBUG:day 5:New rule: Rule(before=97, after=13)
DEBUG:day 5:New rule: Rule(before=97, after=61)
DEBUG:day 5:Added new key=61
DEBUG:day 5:New rule: Rule(before=97, after=61)
DEBUG:day 5:New rule: Rule(before=97, after=47)
DEBUG:day 5:New rule: Rule(before=97, after=47)
DEBUG:day 5:Added new key=75
DEBUG:day 5:New rule: Rule(before=75, after=29)
DEBUG:day 5:Added new key=29
DEBUG:day 5:New rule: Rule(before=75, after=29)
DEBUG:day 5:New rule: Rule(before=61, after=13)
DEBUG:day 5:New rule: Rule(before=61, after=13)
DEBUG:day 5:New rule: Rule(before=75, after=53)
DEBUG:day 5:New rule: Rule(before=75, after=53)
DEBUG:day 5:New rule: Rule(before=29, after=13)
DEBUG:day 5:New rule: Rule(before=29, after=13)
DEBUG:day 5:New rule: Rule(be

In [6]:
# cool now we can look up applicable rules as we check each value
def check_ordering(update: list[int], rules: dict[int: Rule]) -> bool:
    """returns true if the ordering is correct"""
    log.info(f'checking {update}')
    for index, number in enumerate(update):
        log.debug(f'checking {number=} at {index=}')
        for rule in rules[number]:
            log.debug(f'checking {rule}')
            if rule.before == number:
                if rule.after in update[:index]:
                    log.info(f'BAD: {rule.after} found before {rule.before}!')
                    return False
            else:
                if rule.before in update[index + 1:]:
                    log.info(f'BAD: {rule.before} found after {rule.after}!')
                    return False
            log.debug('GOOD: Rule ok')
    else:
        log.info('GOOD: passed all rule checks')
        return True

log.setLevel(logging.INFO)
example_total = 0
for update in example_updates:
    if check_ordering(update, example_rules):
        example_total += update[len(update) // 2]

log.info(f'{example_total=}')

INFO:day 5:checking [75, 47, 61, 53, 29]
INFO:day 5:GOOD: passed all rule checks
INFO:day 5:checking [97, 61, 53, 29, 13]
INFO:day 5:GOOD: passed all rule checks
INFO:day 5:checking [75, 29, 13]
INFO:day 5:GOOD: passed all rule checks
INFO:day 5:checking [75, 97, 47, 61, 53]
INFO:day 5:BAD: 97 found after 75!
INFO:day 5:checking [61, 13, 29]
INFO:day 5:BAD: 29 found after 13!
INFO:day 5:checking [97, 13, 75, 29, 47]
INFO:day 5:BAD: 29 found after 13!
INFO:day 5:example_total=143


In [7]:
# ok let's do it for real!
ordering, updates = parse_from_file('day_5.txt', parser)
rules = get_rules(ordering)
log.setLevel(logging.WARNING)
total = 0
for update in updates:
    if check_ordering(update, rules):
        total += update[len(update) // 2]
markdown(
    f'The total middle page number of correctly-ordered rules is: {total}')

INFO:advent_of_code_utils.py:2 items loaded from "day_5.txt"
INFO:day 5:created rules lookup for 49 values!


The total middle page number of correctly-ordered rules is: 7307